In [4]:
import import_files as MrtRecommendationDependencies
from datetime import datetime
from fbprophet import Prophet
import pandas as pd
from ridership_model import RidershipModel
from mrt_ridership_model import MrtRidershipModel
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
from util import Util

In [3]:
df = pd.read_csv(MrtRecommendationDependencies.get_dataset_path("ayala_weather_transformed.csv"), parse_dates=["datetime"], index_col="datetime")

In [4]:
splitter = TimeSeriesSplit(n_splits=10)

In [5]:
splits = splitter.split(df)

In [6]:
dataset = []

In [7]:
for train_index, test_index in splits:
    dataset.append({'train': df.iloc[train_index], 'test': df.iloc[test_index]})

In [8]:
train = dataset[0]['train']
test = dataset[0]['test']

In [9]:
entry_model = RidershipModel('entry')

In [10]:
entry_model.fit(train)

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [11]:
predict = entry_model.predict(test)

In [12]:
test.head()

,entry,exit,temp,humidity,rain_1h,rain_3h,weather_main
datetime,,,,,,,
2018-08-03 09:00:00,603.0,1247.0,28.62,72,0.0,0.0,Haze
2018-08-03 10:00:00,679.0,1223.0,28.04,72,0.0,0.0,Haze
2018-08-03 11:00:00,727.0,1438.0,27.96,75,0.0,0.0,Haze
2018-08-03 12:00:00,932.0,1667.0,27.99,75,0.0,0.0,Haze
2018-08-03 13:00:00,1021.0,1193.0,27.65,75,0.0,0.0,Haze


In [13]:
predict.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Bonifacio Day,Bonifacio Day_lower,Bonifacio Day_upper,Christmas Day,...,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,holiday,lower_window,upper_window,prediction
0,2018-08-03 09:00:00,926.834647,340.863481,1236.263712,926.834647,926.834647,0.0,0.0,0.0,0.0,...,20.216585,20.216585,0.0,0.0,0.0,780.504042,0,0.0,0.0,780.504042
1,2018-08-03 10:00:00,926.821875,212.911899,1100.674417,926.821875,926.821875,0.0,0.0,0.0,0.0,...,-2.701033,-2.701033,0.0,0.0,0.0,628.183892,0,0.0,0.0,628.183892
2,2018-08-03 11:00:00,926.809103,147.581236,1018.066501,926.809103,926.809103,0.0,0.0,0.0,0.0,...,-26.731427,-26.731427,0.0,0.0,0.0,599.222483,0,0.0,0.0,599.222483
3,2018-08-03 12:00:00,926.796332,338.578145,1203.921724,926.796332,926.796332,0.0,0.0,0.0,0.0,...,-51.744630,-51.744630,0.0,0.0,0.0,767.913463,0,0.0,0.0,767.913463
4,2018-08-03 13:00:00,926.783560,597.972092,1453.457810,926.783560,926.783560,0.0,0.0,0.0,0.0,...,-77.597950,-77.597950,0.0,0.0,0.0,1054.348783,0,0.0,0.0,1054.348783


In [14]:
print(r2_score(test['entry'], predict['yhat']))

0.7581499356121717


In [15]:
entry_r2s = []
exit_r2s = []
for row in dataset:
    train = row['train']
    test = row['test']
    entry_model = RidershipModel('entry')
    exit_model = RidershipModel('exit')
    entry_model.fit(train)
    exit_model.fit(train)
    entry_pred = entry_model.predict(test)
    exit_pred = exit_model.predict(test)
    entry_r2 = r2_score(test['entry'], entry_pred['prediction'])
    exit_r2 = r2_score(test['exit'], exit_pred['prediction'])
    entry_r2s.append(entry_r2)
    exit_r2s.append(exit_r2)
    print(f"R2 score for Entry: {entry_r2}")
    print(f"R2 score for Exit: {exit_r2}")
    
    

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


R2 score for Entry: 0.7581499356121717
R2 score for Exit: 0.823823000702068


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


R2 score for Entry: 0.8360394349230043
R2 score for Exit: 0.8388211701206936


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


...

R2 score for Entry: 0.802873146707711
R2 score for Exit: 0.8471000490854046


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


R2 score for Entry: 0.7853319007812596
R2 score for Exit: 0.796684599427945


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


R2 score for Entry: 0.7790069009629359
R2 score for Exit: 0.791890172315921


In [16]:
np.mean(entry_r2s)

0.7812506126313672

In [17]:
np.mean(exit_r2s)

0.7912845358835725

In [18]:
test = dataset[4]['test']

In [19]:
test.head()

,entry,exit,temp,humidity,rain_1h,rain_3h,weather_main
datetime,,,,,,,
2018-12-14 01:00:00,0.0,0.0,24.95,90,0.13,0.0,Rain
2018-12-14 02:00:00,0.0,0.0,27.56,78,0.00,2.5,Rain
2018-12-14 03:00:00,0.0,0.0,27.88,78,0.00,2.5,Rain
2018-12-14 04:00:00,0.0,0.0,27.60,78,0.00,2.5,Rain
2018-12-14 05:00:00,133.0,172.0,27.62,77,0.00,0.0,Clouds


In [24]:
test[test.index > datetime(2019,1,1)].head(20)

,entry,exit,temp,humidity,rain_1h,rain_3h,weather_main
datetime,,,,,,,
2019-01-01 01:00:00,0.0,0.0,25.59,90,0.00,0.00,Clouds
2019-01-01 02:00:00,0.0,0.0,27.32,85,0.00,3.00,Rain
2019-01-01 03:00:00,0.0,0.0,27.87,85,0.00,3.00,Rain
2019-01-01 04:00:00,0.0,0.0,28.26,85,0.00,3.00,Rain
2019-01-01 05:00:00,0.0,0.0,29.27,61,0.00,0.00,Haze
2019-01-01 06:00:00,181.0,29.0,28.52,61,0.00,0.00,Haze
2019-01-01 07:00:00,333.0,266.0,28.45,61,0.00,0.00,Haze
2019-01-01 08:00:00,237.0,226.0,27.94,67,0.00,0.00,Haze
2019-01-01 09:00:00,243.0,417.0,27.58,67,0.00,0.00,Haze


In [5]:
model = MrtRidershipModel()

NameError: name 'init_models' is not defined